# Import Tensorflow

In [0]:
!pip install tensorflow==2.0.0b1

# SPIT Imports

In [0]:
# imports
import numpy as np
import os

from importlib import reload
from pkg_resources import resource_filename
from google.colab import drive
import imageio

In [0]:
import tensorflow
tensorflow.__version__

'2.0.0-beta1'

In [0]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
!git clone https://github.com/pypeit/spit.git

fatal: destination path 'spit' already exists and is not an empty directory.


In [0]:
%cd spit

/content/spit/spit


In [0]:
!git checkout x_fussing

Already on 'x_fussing'
Your branch is up to date with 'origin/x_fussing'.


In [0]:
!git pull

Already up to date.


In [0]:
!python setup.py develop

python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [0]:
# imports
from importlib import reload

from spit import image_loader
from spit import labels
from spit import preprocess
from spit import classifier
from spit import utils

# Build DenseNet

In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
def dense_block(model):
  
  # 1x1 convolution bottleneck
  
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.ReLU())
  #### HYPERPARAMETER ####
  model.add(keras.layers.Conv2D(4*filters, kernel_size=1))
  model.add(keras.layers.Dropout(rate=dropout_rate))
  
  # 3x3 dense convolution
  
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.ReLU())
  #### HYPERPARAMETER ####
  model.add(keras.layers.Conv2D(filters, kernel_size=3))
  #### HYPERPARAMETER ####
  model.add(keras.layers.Dropout(rate=dropout_rate))
  
  return

def transition_layer(model):
#   x = Batch_Normalization(x, training=self.training, scope=scope+'_batch1')
#   x = Relu(x)
  model.add(keras.layers.Conv2D(filters, kernel_size=1))
  model.add(keras.layers.AveragePooling2D(pool_size=(2,2), strides=2))
#   x = conv_layer(x, filter=self.filters, kernel=[1,1], layer_name=scope+'_conv1')
#   x = Drop_out(x, rate=dropout_rate, training=self.training)
#   x = Average_pooling(x, pool_size=[2,2], stride=2)
  return
  

In [0]:
#### DENSE NET ARCHITECTURE #####
#
# CONV --> BLOCK --> TRANSITION --> BLOCK --> TRANSITION --> BLOCK --> POOL --> SOFTMAX
#
# BLOCK:
# BatchNorm --> ReLu --> Conv (1x1) --> DropOut --> BatchNorm --> ReLu --> Conv (3x3) --> DropOut --> output
#

model = keras.Sequential()

# hyperparameters
filters = 64
dropout_rate = .5

# add a conv layer
model.add(keras.layers.Conv2D(filters, kernel_size=7, strides=(2, 2), padding='valid', activation='relu', 
                            input_shape = (210, 650, 1)))
# add block
dense_block(model)
# add transition
transition_layer(model)
# add block
dense_block(model)
# add transition
transition_layer(model)
# add block
dense_block(model)
# add pool
model.add(keras.layers.GlobalAveragePooling2D())
# add flatten
model.add(keras.layers.Flatten())

# Produce 0-1 probabilities with softmax
model.add(keras.layers.Dense(5, activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Fake Data

In [0]:
x_train = np.random.random((100, 210, 650, 1))
train_labels = np.random.randint(5, size=(100, 1))

In [0]:
y_train = keras.utils.to_categorical(train_labels, num_classes=5)

In [0]:
 model.fit(
        x_train, 
        y_train, 
        epochs=5, 
        batch_size=6 
 )

Train on 100 samples
Epoch 1/5
 12/100 [==>...........................] - ETA: 2:54 - loss: 2.8118 - accuracy: 0.2500

KeyboardInterrupt: ignored

# Kast Training

In [0]:
  os.environ['SPIT_PATH'] = "/content/drive/My Drive/Colab Notebooks/SPIT_DATA"
  os.environ['SAVE_PATH'] = "/content/drive/My Drive/Colab Notebooks/"

  # might need to add subset
  def _train(epochs, batch_size, subset_percent=None, train_images=None, train_labels=None, validation_data=None, steps_per_epoch=None, validation_freq=1, test_model=None, spit_path=os.getenv('SPIT_PATH'), save_path=os.getenv('SAVE_PATH')):
    """

    Trains the classifier with given images, labels, and training parameters.

    Parameters:

    :param epochs:
      Number of epochs of the training. 
      Must be an integer value.

    :param batch_size:
      Size of the training batches formed in the process. 
      Must be an integer value.

    :param save_path:
      Path to where the best model will be saved.

    :param train_images:
       Set of images for model to train on.
       Assume this is a numpy array with (batch_size, width, height, num_channels) as its dimensions.

    :param train_labels:
      Set of test labels corresponding to test images.
      Assume this is a rank 1 array with (batch_size, ) as its dimensions.

    :param validation_data:
      Data to be used for the validation set. 
      Assume this is a tuple with (images, labels) with same dimensions as train_images, train_labels.
      If None is specified, validation_data will be None.

    :param steps_per_epoch:
      Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch.
      Assume this is an integer value. If None is specified, this will be None.  

    :param validation_freq:
      Specifies how many training epochs to run before a new validation run is performed.
      Assume this is an integer value or a collection containing the epochs at which to run validation (ie [1,2,10]).

    :param test_model:
      An alternative choice for model to train on. Assume None.

    :param spit_path & param save_path:
      Path to the spit images and where the model will be saved respectively.
      ***Environmental variables must be set by caller or the path must be passed manually.***

    Returns:
    :returns history:
      Tensorflow History object containing loss and accuracy data over the training.

    """
    # choose the model
    if test_model is None:
      model = test_model
    else:
      model = test_model

    # if None is passed, then use the kast images
    if train_images is None or train_labels is None:
      # load training set
      train = np.load(os.path.join(spit_path, 'Kast', 'kast_train.npz'))
      train_images = train['images']
      train_labels = train['labels']

      # load validation set
      validate = np.load(os.path.join(spit_path, 'Kast', 'kast_validate.npz'))
      v_images = validate['images']
      v_labels = validate['labels']

      validation_data = (v_images, v_labels)

    # change to categorical and make subsets
    if validation_data is not None:
      valid_images, valid_labels = validation_data
      if subset_percent is not None:
        valid_images, valid_labels = split_array(valid_images, valid_labels, subset_percent)
      valid_labels = keras.utils.to_categorical(valid_labels, num_classes=len(label_dict))
      validation_data = (valid_images, valid_labels)

    if train_images is not None and train_labels is not None:
      if subset_percent is not None:
        train_images, train_labels = split_array(train_images, train_labels, subset_percent)
      train_labels = keras.utils.to_categorical(train_labels, num_classes=len(label_dict))

    # checkpoint to track best model
    checkpoint=keras.callbacks.ModelCheckpoint(save_path+'best_model.h5', monitor='val_acc', save_best_only=True, mode='max')

    # train the model
    history = model.fit(
          train_images, 
          train_labels, 
          epochs=epochs, 
          batch_size=batch_size,
          validation_data=validation_data,
          steps_per_epoch=steps_per_epoch,
          validation_freq=validation_freq,
          callbacks=[checkpoint]
    )
    # loss and accuracy data
    keys = history.history
    # save to disc differently based on whether validation set was used
#     if len(history.history.keys()) == 2: # can we abstract this away?
#       np.savez_compressed('history.npz', loss=keys['loss'], acc=keys['acc']) #can these keys be abstracted away?
#     else:
#       np.savez_compressed('history.npz', loss=keys['loss'], acc=keys['acc'], val_loss=keys['val_loss'], val_acc=keys['val_acc'])

    return history

  import math

  def split_array(images, labels, subset_percent):
    """
    Splits dataset based on a percentage value.

    Parameters:
    :param images:
      Images from a dataset to be trained on.
      4-D Numpy array with (batch_size, width, height, num_channels) as its dimensions.

    :param labels:
      Labels from a dataset to be trained on.
      Rank 1 Numpy array with (batch_size,) as its dimensions.

    :param subset_percent:
      Float value determining percentage of subset to remain.

    Returns:
    :returns split_images:
      Numpy array containing a fraction of the initial images parameter (batch_size*subset_percent)

    :returns split_labels:
      Numpy array containing a fraction of the initial labels parameter (batch_size*subset_percent)
    """
    split_images = []
    split_labels = []
    
    # get all unique labels
    uni_lbls = np.unique(labels)
    
    # find all instances of labels and subset based on that
    for uni_lbl in uni_lbls:
      idx = np.where(labels==uni_lbl)[0]
      # 0 : len(idx)*subset_percent
      lower = 0
      upper = int(math.floor(len(idx)*subset_percent))
      split_images.extend(images[idx[lower:upper]])
      split_labels.extend(labels[idx[lower:upper]])
    return np.asarray(split_images), np.asarray(split_labels)

In [0]:
label_dict = labels.kast_label_dict()
preproc_dict = preprocess.original_preproc_dict()
classify_dict = labels.kast_classify_dict(label_dict)

In [0]:
history = _train(epochs=5, batch_size=6, subset_percent = .05, test_model=model)

Train on 775 samples, validate on 190 samples
Epoch 1/5
774/775 [============================>.] - ETA: 1s - loss: 1.1711 - accuracy: 0.5323 

W0724 20:52:36.457918 140551106664320 callbacks.py:986] Can save best model only with val_acc available, skipping.


775/775 [==============================] - 1498s 2s/sample - loss: 1.1718 - accuracy: 0.5316 - val_loss: 12.1867 - val_accuracy: 0.2000
Epoch 2/5
774/775 [============================>.] - ETA: 1s - loss: 0.7773 - accuracy: 0.7003 

W0724 21:17:27.445801 140551106664320 callbacks.py:986] Can save best model only with val_acc available, skipping.


775/775 [==============================] - 1491s 2s/sample - loss: 0.7765 - accuracy: 0.7006 - val_loss: 0.8673 - val_accuracy: 0.6842
Epoch 3/5
774/775 [============================>.] - ETA: 1s - loss: 0.7296 - accuracy: 0.7674 

...

W0724 22:07:09.766519 140551106664320 callbacks.py:986] Can save best model only with val_acc available, skipping.


775/775 [==============================] - 1492s 2s/sample - loss: 0.6113 - accuracy: 0.7794 - val_loss: 1.0964 - val_accuracy: 0.6947
Epoch 5/5
774/775 [============================>.] - ETA: 1s - loss: 0.5828 - accuracy: 0.7855 

W0724 22:31:57.023474 140551106664320 callbacks.py:986] Can save best model only with val_acc available, skipping.


775/775 [==============================] - 1487s 2s/sample - loss: 0.5828 - accuracy: 0.7858 - val_loss: 3.1274 - val_accuracy: 0.5368


KeyError: ignored

In [0]:
utils.display_training_trends(history, key1='acc', key2='val_acc')

NameError: ignored